# Fine-tuning with Triplet Loss

Learn to use triplet loss for metric learning and semantic similarity.

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from sentence_transformers.evaluation import TripletEvaluator
from torch.utils.data import DataLoader
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

## What is Triplet Loss?

Triplet loss learns embeddings by comparing:
- **Anchor**: Reference text
- **Positive**: Similar text (should be close)
- **Negative**: Different text (should be far)

**Goal**: `distance(anchor, positive) < distance(anchor, negative) + margin`

```
Anchor: "How to reset password?"
Positive: "Click forgot password link"  ← Should be CLOSE
Negative: "Pricing plans start at $29"  ← Should be FAR
```

## Load Triplet Training Data

In [ ]:
# Load hard triplets (500 examples with challenging negatives)
with open('../fixtures/input/training_triplets_hard.json', 'r') as f:
    triplets = json.load(f)

print(f"Loaded {len(triplets)} hard triplet examples\n")

# Show example
example = triplets[0]
print("Triplet structure:")
print(f"  Anchor:   {example['anchor']}")
print(f"  Positive: {example['positive']}")
print(f"  Negative: {example['negative']}")

print("\n💡 Hard negatives are semantically similar but not relevant")
print("   This makes training more challenging and effective!")

## Visualize Generic Model Behavior

In [ ]:
# Load generic model
generic_model = SentenceTransformer('all-MiniLM-L6-v2')

# Test on sample triplet
triplet = triplets[0]
anchor_emb = generic_model.encode(triplet['anchor'])
positive_emb = generic_model.encode(triplet['positive'])
negative_emb = generic_model.encode(triplet['negative'])

# Calculate distances
pos_sim = cosine_similarity([anchor_emb], [positive_emb])[0][0]
neg_sim = cosine_similarity([anchor_emb], [negative_emb])[0][0]

print("Generic model similarities:")
print(f"  Anchor ↔ Positive: {pos_sim:.3f}")
print(f"  Anchor ↔ Negative: {neg_sim:.3f}")
print(f"  Margin (pos - neg): {(pos_sim - neg_sim):.3f}")

if pos_sim - neg_sim < 0.2:
    print("\n⚠️  Margin too small - model doesn't distinguish well!")

## Prepare Triplet Training Examples

In [ ]:
# Create InputExample objects for triplets
train_examples = []

for item in triplets:
    train_examples.append(
        InputExample(
            texts=[item['anchor'], item['positive'], item['negative']]
        )
    )

print(f"Created {len(train_examples)} triplet training examples")
print(f"\nExample format: {len(train_examples[0].texts)} texts per triplet")

## Split Train/Validation

In [ ]:
# Split 80/20 for train/validation
split_idx = int(0.8 * len(train_examples))
train_data = train_examples[:split_idx]
val_data = train_examples[split_idx:]

print(f"Training examples: {len(train_data)}")
print(f"Validation examples: {len(val_data)}")

## Create DataLoader with Triplet Loss

In [ ]:
# Create DataLoader
train_dataloader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=16
)

print(f"DataLoader created:")
print(f"  Batch size: 16")
print(f"  Number of batches: {len(train_dataloader)}")

## Configure Triplet Loss

**TripletLoss parameters:**
- `distance_metric`: How to measure distance (cosine, euclidean)
- `triplet_margin`: Minimum separation between positive and negative

In [ ]:
# Load model to fine-tune
model = SentenceTransformer('all-MiniLM-L6-v2')

# Define triplet loss with margin
train_loss = losses.TripletLoss(
    model=model,
    distance_metric=losses.TripletDistanceMetric.COSINE,
    triplet_margin=0.5  # Positive should be 0.5 closer than negative
)

print("✓ Loss function: TripletLoss")
print("  Distance metric: Cosine")
print("  Margin: 0.5")

## Setup Evaluator

In [ ]:
# Create evaluator from validation data
anchors = [ex.texts[0] for ex in val_data]
positives = [ex.texts[1] for ex in val_data]
negatives = [ex.texts[2] for ex in val_data]

evaluator = TripletEvaluator(
    anchors=anchors,
    positives=positives,
    negatives=negatives,
    name='triplet_eval'
)

print(f"✓ Evaluator created with {len(val_data)} validation triplets")

## Fine-tune with Triplet Loss

In [ ]:
# Training configuration
num_epochs = 4  # Reduced for 500 examples - more data needs fewer epochs
warmup_steps = int(0.1 * len(train_dataloader) * num_epochs)
eval_steps = max(50, len(train_dataloader) // 4)  # Evaluate 4 times per epoch

print(f"Training configuration:")
print(f"  Total examples: {len(train_data)} (hard negatives)")
print(f"  Epochs: {num_epochs}")
print(f"  Warmup steps: {warmup_steps}")
print(f"  Evaluation frequency: every {eval_steps} steps")
print(f"  Total training steps: {len(train_dataloader) * num_epochs}")

# Evaluate before training
print("\n" + "="*60)
print("INITIAL EVALUATION (before training)")
print("="*60)
initial_result = evaluator(model, output_path='../output/triplet_finetuned_model')
# TripletEvaluator returns dict like {'triplet_eval_cosine_accuracy': 0.98}
if isinstance(initial_result, dict):
    initial_score = list(initial_result.values())[0]
else:
    initial_score = initial_result
print(f"Initial Accuracy: {initial_score:.4f}")
print(f"  (1.0 = perfect, 0.5 = random)")
print("="*60 + "\n")

# Fine-tune
print("Starting fine-tuning with triplet loss on HARD examples...")
print("Watch for evaluation outputs below:\n")

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    evaluator=evaluator,
    evaluation_steps=eval_steps,  # Evaluate more frequently
    output_path='../output/triplet_finetuned_model',
    show_progress_bar=True,
    save_best_model=True,  # Only save best model
    use_amp=False,  # Disable mixed precision for stability
    checkpoint_save_steps=10000,  # Don't save intermediate checkpoints
    checkpoint_save_total_limit=1,  # Keep only best model
    optimizer_params={'lr': 2e-5}  # Learning rate for better convergence
)

print("\n" + "="*60)
print("✓ Fine-tuning complete!")
print("="*60)

# Load and display final metrics
import pandas as pd
import os

# NOTE: sentence-transformers saves evaluation results to 'eval/' subfolder
# with prefix from evaluator class name (e.g., 'triplet_evaluation_')
metrics_file = '../output/triplet_finetuned_model/eval/triplet_evaluation_triplet_eval_results.csv'
if os.path.exists(metrics_file):
    print("\n" + "="*60)
    print("TRAINING METRICS SUMMARY")
    print("="*60)
    metrics_df = pd.read_csv(metrics_file)
    
    # Filter out pre-training evaluation (epoch=-1, steps=-1)
    metrics_df = metrics_df[(metrics_df['epoch'] >= 0) & (metrics_df['steps'] >= 0)]
    
    # Show first, middle, and last few rows
    print("\nFirst evaluations:")
    print(metrics_df.head(3).to_string(index=False))
    if len(metrics_df) > 6:
        print("\n...")
        print("\nFinal evaluations:")
        print(metrics_df.tail(3).to_string(index=False))
    
    print("\n" + "="*60)
    # Use accuracy_cosine column instead of 'accuracy'
    metric_col = 'accuracy_cosine' if 'accuracy_cosine' in metrics_df.columns else 'accuracy'
    best_acc = metrics_df[metric_col].max()
    best_step = metrics_df.loc[metrics_df[metric_col].idxmax(), 'steps']
    improvement = best_acc - initial_score
    improvement_pct = (improvement / initial_score * 100) if initial_score > 0 else 0
    
    print(f"📊 RESULTS:")
    print(f"  Initial Accuracy:  {initial_score:.4f}")
    print(f"  Best Accuracy:     {best_acc:.4f} (at step {int(best_step)})")
    print(f"  Improvement:       +{improvement:.4f} ({improvement_pct:+.1f}%)")
    print("="*60)
    
    # Note about training loss
    print("\n💡 NOTE: 'No log' in Training/Validation Loss columns is NORMAL.")
    print("   sentence-transformers logs evaluation metrics (accuracy)")
    print("   but doesn't log raw loss values in the progress bar.")
    print("   The evaluation metrics are saved to CSV and shown above.")
    
    if improvement > 0.05:
        print("\n✅ Model successfully improved on hard negatives!")
    elif improvement > 0:
        print("\n⚠️  Small improvement - hard negatives are challenging")
    else:
        print("\n⚠️  No improvement - may need more epochs or different hyperparameters")
else:
    print("\n⚠️  Metrics file not found at:")
    print(f"   {metrics_file}")
    print("   Check that the evaluator and output_path are correctly configured")

## Visualize Training Progress

In [ ]:
# Plot training progress
import pandas as pd
import os

metrics_file = '../output/triplet_finetuned_model/eval/triplet_evaluation_triplet_eval_results.csv'
if os.path.exists(metrics_file):
    metrics_df = pd.read_csv(metrics_file)
    
    # Filter out pre-training evaluation (epoch=-1, steps=-1)
    metrics_df = metrics_df[(metrics_df['epoch'] >= 0) & (metrics_df['steps'] >= 0)]
    
    # Use accuracy_cosine column
    metric_col = 'accuracy_cosine' if 'accuracy_cosine' in metrics_df.columns else 'accuracy'
    
    fig, ax = plt.subplots(figsize=(12, 5))
    
    ax.plot(metrics_df['steps'], metrics_df[metric_col], marker='o', linewidth=2, markersize=6)
    ax.axhline(initial_score, color='red', linestyle='--', label=f'Initial: {initial_score:.4f}')
    ax.set_xlabel('Training Steps', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title('Triplet Loss Training Progress', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Annotate best score
    best_idx = metrics_df[metric_col].idxmax()
    best_step = metrics_df.loc[best_idx, 'steps']
    best_acc = metrics_df.loc[best_idx, metric_col]
    ax.annotate(f'Best: {best_acc:.4f}', 
                xy=(best_step, best_acc),
                xytext=(10, -20),
                textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nAccuracy improved from {initial_score:.4f} to {best_acc:.4f}")
    print(f"Total improvement: +{(best_acc - initial_score):.4f} ({((best_acc - initial_score) / initial_score * 100):.1f}%)")
else:
    print("⚠️  No metrics file found. Training may not have completed or evaluator didn't run.")
    print(f"   Expected path: {metrics_file}")

## Compare Before/After

In [ ]:
# Load fine-tuned model
finetuned_model = SentenceTransformer('../output/triplet_finetuned_model')

# Test on multiple triplets
test_triplets = triplets[split_idx:split_idx+10]  # Use validation examples

generic_margins = []
finetuned_margins = []

for triplet in test_triplets:
    # Generic model
    anchor_emb = generic_model.encode(triplet['anchor'])
    pos_emb = generic_model.encode(triplet['positive'])
    neg_emb = generic_model.encode(triplet['negative'])
    
    pos_sim = cosine_similarity([anchor_emb], [pos_emb])[0][0]
    neg_sim = cosine_similarity([anchor_emb], [neg_emb])[0][0]
    generic_margins.append(pos_sim - neg_sim)
    
    # Fine-tuned model
    anchor_emb = finetuned_model.encode(triplet['anchor'])
    pos_emb = finetuned_model.encode(triplet['positive'])
    neg_emb = finetuned_model.encode(triplet['negative'])
    
    pos_sim = cosine_similarity([anchor_emb], [pos_emb])[0][0]
    neg_sim = cosine_similarity([anchor_emb], [neg_emb])[0][0]
    finetuned_margins.append(pos_sim - neg_sim)

# Statistics
print("Margin Statistics (positive - negative similarity):")
print("\nGeneric Model:")
print(f"  Mean margin: {np.mean(generic_margins):.3f}")
print(f"  Std deviation: {np.std(generic_margins):.3f}")
print(f"  Min margin: {np.min(generic_margins):.3f}")

print("\nFine-tuned Model:")
print(f"  Mean margin: {np.mean(finetuned_margins):.3f}")
print(f"  Std deviation: {np.std(finetuned_margins):.3f}")
print(f"  Min margin: {np.min(finetuned_margins):.3f}")

improvement = np.mean(finetuned_margins) - np.mean(generic_margins)
print(f"\n✓ Average margin improvement: +{improvement:.3f}")

## Visualize Margin Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram comparison
axes[0].hist(generic_margins, alpha=0.5, label='Generic', bins=15)
axes[0].hist(finetuned_margins, alpha=0.5, label='Fine-tuned', bins=15)
axes[0].axvline(0, color='red', linestyle='--', label='No separation')
axes[0].set_xlabel('Margin (positive - negative similarity)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Margin Distribution')
axes[0].legend()

# Individual comparisons
x = np.arange(len(generic_margins))
width = 0.35
axes[1].bar(x - width/2, generic_margins, width, label='Generic', alpha=0.7)
axes[1].bar(x + width/2, finetuned_margins, width, label='Fine-tuned', alpha=0.7)
axes[1].axhline(0, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Test Example')
axes[1].set_ylabel('Margin')
axes[1].set_title('Per-Example Margins')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📊 Higher margins = Better separation between relevant and irrelevant")

## Test on Real Queries

In [ ]:
# Test on new queries
test_cases = [
    {
        "query": "password recovery process",
        "relevant": "Click forgot password and follow reset instructions",
        "irrelevant": "Annual subscription includes priority support"
    },
    {
        "query": "explain machine learning concepts",
        "relevant": "ML algorithms learn patterns from data automatically",
        "irrelevant": "Our office is open Monday through Friday"
    },
    {
        "query": "API integration steps",
        "relevant": "Add Bearer token to Authorization header for API calls",
        "irrelevant": "Employees receive 15 vacation days per year"
    }
]

print("Real Query Testing:\n")

for i, test in enumerate(test_cases, 1):
    print(f"Test {i}: {test['query']}")
    
    # Generic model
    q_emb = generic_model.encode(test['query'])
    rel_emb = generic_model.encode(test['relevant'])
    irr_emb = generic_model.encode(test['irrelevant'])
    gen_margin = (
        cosine_similarity([q_emb], [rel_emb])[0][0] - 
        cosine_similarity([q_emb], [irr_emb])[0][0]
    )
    
    # Fine-tuned model
    q_emb = finetuned_model.encode(test['query'])
    rel_emb = finetuned_model.encode(test['relevant'])
    irr_emb = finetuned_model.encode(test['irrelevant'])
    ft_margin = (
        cosine_similarity([q_emb], [rel_emb])[0][0] - 
        cosine_similarity([q_emb], [irr_emb])[0][0]
    )
    
    print(f"  Generic margin: {gen_margin:.3f}")
    print(f"  Fine-tuned margin: {ft_margin:.3f}")
    print(f"  Improvement: +{(ft_margin - gen_margin):.3f}\n")

## Analyze Embedding Space

In [ ]:
from sklearn.decomposition import PCA

# Select sample texts from different categories
sample_texts = [
    "How to reset my password?",
    "Password recovery steps",
    "Login issues troubleshooting",
    "What is machine learning?",
    "Explain deep learning",
    "Neural networks introduction",
    "Professional plan pricing",
    "Subscription costs",
    "Payment options"
]

categories = ['password'] * 3 + ['ml'] * 3 + ['pricing'] * 3

# Encode with both models
generic_embs = generic_model.encode(sample_texts)
finetuned_embs = finetuned_model.encode(sample_texts)

# Reduce to 2D
pca = PCA(n_components=2)
generic_2d = pca.fit_transform(generic_embs)
finetuned_2d = pca.fit_transform(finetuned_embs)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

colors = {'password': 'red', 'ml': 'blue', 'pricing': 'green'}

for ax, data, title in zip(
    axes, 
    [generic_2d, finetuned_2d], 
    ['Generic Model', 'Fine-tuned Model']
):
    for i, (cat, text) in enumerate(zip(categories, sample_texts)):
        ax.scatter(data[i, 0], data[i, 1], c=colors[cat], s=100, alpha=0.6)
        ax.annotate(
            text[:20] + '...' if len(text) > 20 else text,
            (data[i, 0], data[i, 1]),
            fontsize=8,
            alpha=0.7
        )
    
    ax.set_title(title)
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor=colors[cat], label=cat) 
                      for cat in ['password', 'ml', 'pricing']]
    ax.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

print("\n📊 Fine-tuned model should show tighter clusters")

## Hard Negative Mining (Advanced)

For better training, mine hard negatives - examples that are close but wrong.

In [ ]:
# Example: Find hard negatives
query = "What is machine learning?"
positive = "ML learns patterns from data automatically"

# Candidate negatives
candidates = [
    "Deep learning uses neural networks",  # Hard negative (related topic)
    "Professional plan costs $99/month",   # Easy negative (unrelated)
    "NLP processes human language",        # Hard negative (related field)
]

# Encode
q_emb = finetuned_model.encode(query)
p_emb = finetuned_model.encode(positive)
c_embs = finetuned_model.encode(candidates)

# Find similarities
pos_sim = cosine_similarity([q_emb], [p_emb])[0][0]
print(f"Query ↔ Positive: {pos_sim:.3f}\n")

print("Candidate negatives:")
for cand, c_emb in zip(candidates, c_embs):
    sim = cosine_similarity([q_emb], [c_emb])[0][0]
    difficulty = "HARD" if sim > 0.5 else "EASY"
    print(f"  [{sim:.3f}] {difficulty}: {cand}")

print("\n💡 Hard negatives are more informative for training!")

## Summary

✅ Loaded 100 triplet examples (anchor, positive, negative)  
✅ Used TripletLoss with cosine distance and 0.5 margin  
✅ Evaluated on validation set during training  
✅ Achieved larger margins between relevant and irrelevant  
✅ Visualized embedding space improvements  
✅ Learned about hard negative mining  

**Key learnings:**
- Triplet loss explicitly pushes apart dissimilar examples
- Margin parameter controls minimum separation
- Works well with 100+ triplets
- Hard negatives (similar but wrong) improve training
- Better clustering in embedding space

**When to use Triplet Loss:**
- Face recognition / image similarity
- Recommendation systems
- Semantic search with clear negatives
- Metric learning tasks

**Next:** Learn contrastive loss and compare all approaches!